In [ ]:
# Import libraries
import numpy as np
import xarray as xr
import pandas as pd

In [14]:
# Specify filepaths and filenames.
filepath = '/home/jovyan/shared-public/MLinGlaciology/BYOP/data/point_mb_glaciers_norway/'
filename = 'stake_mb_norway_cleaned_ids.csv'

In [17]:
# Load data.
data = pd.read_csv(filepath + filename, sep=';')

data
data.columns

Index(['RGIID', 'GLIMSID', 'BREID', 'utm_zone', 'utm_east_approx',
       'utm_north_approx', 'altitude_approx', 'location_description',
       'location_id', 'stake_no', 'altitude', 'utm_east', 'utm_north',
       'balance_winter', 'balance_summer', 'balance_netto',
       'dt_prev_year_min_date', 'dt_curr_year_max_date',
       'dt_curr_year_min_date', 'stake_remark', 'flag_correction',
       'approx_loc', 'approx_altitude', 'diff_north', 'diff_east',
       'diff_altitude', 'diff_netto'],
      dtype='object')